# How the Anoncred format handles selective disclosure

## Introduction 
This text is aimed at those who seek to understand how the ZKP-CL attestation format (Anoncreds) enables a holder to select which attributes to reveal to a verifier. This text focuses only on the cryptographic protection of attribute disclosures. Possible linking via data or communication analysis is a different matter.

Selective disclosure and the Anoncred format requires a signature scheme that supports signing multiple messages with a single signature. The Camenisch-Lysyanskaya (CL) signature scheme is a multi message capable signature scheme that was developed in a series of papers ([CL01](https://eprint.iacr.org/2001/019.pdf), [CL02](https://cs.brown.edu/people/alysyans/papers/camlys02.pdf), [CL03](http://cs.brown.edu/people/alysyans/papers/camlys02b.pdf), [CL04](https://www.iacr.org/archive/crypto2004/31520055/cl04.pdf)). To understand CL signatures, it is important to understand both the [Discrete Log Problem](https://www.doc.ic.ac.uk/~mrh/330tutor/ch06s02.html#:~:text=The%20discrete%20logarithm%20problem%20is,logarithms%20depends%20on%20the%20groups.) and how [commitment schemes](https://en.wikipedia.org/wiki/Commitment_scheme) work.

The example herein is developed using mainly [CL01](https://eprint.iacr.org/2001/019.pdf). As such, the example differs from the way Anoncreds v 0.7 handles selective disclosure (cf. section 4.7.4 in Lodder, Zundel, and Khovratovich, 2019). To simplify the illustration of the selective disclosure mechanism, several steps outlined in 4.7.4 are missing. Specifically, the example below:

* does not use a nonce
* assumes that all attributes exist in a single attestation
* ignores the setup of the non interactive parts of the proof
* only considers the case where the attribute is fully disclosed (as opposed to disclosing statements about the attribute value, which Anoncreds is capable of)
* uses a simpler proof verification as a consequence of the points above

## Strong RSA based CL signatures

The CL signatures used in the early Anoncred format were based on the strong RSA assumption (cf. [Wiki on SRSA](https://en.wikipedia.org/wiki/Strong_RSA_assumption)). The actors and their interactions are as follows ([source](https://www.csc.kth.se/~buc/PPC/Slides/jan.pdf)):

* An attestation contains $L$ attributes represented as $m_0,\ldots,m_{L-1}$
* An issuer signs a block of messages
* The holder presents the messages, or a subset thereof, to a verfier together with the signature
* The verifier checks the messages and the signature

Steps:

* Generate $n=pq$
* $R_0, \ldots, R_{L-1}, S, Z \in \mathbf{QR}_n$ (ie., generates $L+2$ random numbers from a [quadratic residue](https://en.wikipedia.org/wiki/Quadratic_residue) modulo $n$)
* Public key will be $(n,R_0, \ldots, R_{L-1}, S, Z)$ 
* Secret key is $p$
* Message space is $\{(m_0, \ldots, m_{L-1}): m_i \in \pm \{0,1\}^{l_m}\}$
* Signing algorithm: on input $(m_0, \ldots, m_{L-1})$ chose a random prime $e>l_m + 2$ and a random number $v$ of length $l_v = l_n + l_m + l_r$ where $l_r$ is a security parameter
* Compute the value $A$ s.t. 

\begin{align}
  A = \Bigg(\frac{Z}{R_0^{m_0} \cdot \ldots \cdot R_{L-1}^{m_{L-1}} \cdot S^v}\Bigg)^{\frac{1}{e}} \pmod n
\end{align}

* The signature on the message $(m_0, \ldots, m_{L-1})$ consists of $(e,A,v)$

Verification of the signature on the message is done through:

* $Z \equiv A^e \cdot R_0^{m_0} \cdot \ldots \cdot R_{L-1}^{m_{L-1}} \cdot S^v \pmod n$

## Selective disclosure with CL signatures

The holder can selectively disclose a message by:

1. sending $m_i$ to the verifier, who computes $(R_i^{m_i}) \pmod n$
2. compute $(R_i^{m_i}) \pmod n$, which is the commitment of $m_i$ and sent the commitment to he verifier

The holder can send over the messages they wish to fully disclose, and the commitment for the messages they wish to keep secret.

In [1]:
from Crypto.Util import number

In [2]:
# In this example we use two messages

# For demo purposes to make print more readable
p = number.getPrime(32)
q = number.getPrime(32)

# p = number.getStrongPrime(512)
# q = number.getStrongPrime(512)

# The public parts
n = p*q
R_0 = pow(number.getRandomRange(2, n), 2, n)
R_1 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)

# The two messages
m_0 = number.getRandomRange(2, n)
m_1 = number.getRandomRange(2, n)

# The signature related parts
e = number.getPrime(m_0.bit_length() + 1) # Part of signature
e_inv = pow(e, -1, (p-1)*(q-1))
v = number.getRandomNBitInteger(m_0.bit_length() + n.bit_length() + 1) # Part of signature

commitment_vector = pow(R_0, m_0, n) * pow(R_1, m_1, n) * pow(S, v, n)
commitment_vector_inv = pow(commitment_vector, -1, n)

A = pow((Z * commitment_vector_inv) % n, e_inv, n) # Part of signature

print(f'Public key is:\nn = {n}\n(R_0, R_1) = {(R_0, R_1)}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')
print(f'\nMessage (m_0,m_1) is ({m_0}, {m_1}) \nSignature on (m_0,m_1) is: e = {e}, s = {A}, v = {v}')

print('\nRHS is the same as LHS:', (pow(A, e, n) * commitment_vector) % n == Z)

Public key is:
n = 8702221940308231207
(R_0, R_1) = (4389054633200036254, 7817933546012456372)
S = 3145382326342893839
Z = 1229536015517545026

Private key is: (3746994661, 2322453787)

Message (m_0,m_1) is (5246371423628340082, 5889522426947138268) 
Signature on (m_0,m_1) is: e = 11710050195370459463, s = 4005833718575318972, v = 120376054328941101764546907692512386001

RHS is the same as LHS: True


## Summary and concluding thoughts

As aforementioned, the example above illustrates selective disclosure using only fully disclosed attributes. Anoncred is capable of more advanced disclosure types, but these are more complex and not strictly necessary to show in order to provide a basic understanding of how Anoncred achieves selective disclosure.

**What Anoncred does** 

* Anoncred supports selective disclosure of various types (above the only one demonstrated was the fully revealed type of disclosure)
    * Fully hidden: This is a ZKP of knowledge, ie., the only thing revealed about the attribute is that the holder knows the value.
    * Linked: The holder reveals that two attributes have the same value
    * Partially disclosed: The holder reveals some predicate about an attribute value (Anoncreds relies on Bulletproofs)
    * Fully revealed: The attribute value is shared
* the issuer returns a partially signed attestation (using a blind signature scheme) and it is the holder that completes the signature. Consequently, the issuer does not know the final signature value.
* Provides very high privacy and user control. A user is in absolute and sole control over their identity related data.
* Works even in adversarial environments with colluding actors.

**How Anoncred does it**

* BLINDING: Holder blinded attributes are one of three types of attributes in Anoncred
* ANTI-CORRELATION: Anoncred supports both issuer unlinkability (a presentation cannot be linked to a specific credential issuance) and verifier unlinkability (two presentations cannot be linked to each other).
* BINDING: Mainly through the use of a blinded attribute
* ZKP: Used extensively but not shown herein.
* GROUPING LOGIC: A bit commitment vector.

**Potential concerns**

1. Complex.
2. Not a standard.